In [ ]:
import utils
import numpy as np
import cv2
import open3d as o3d
import time

In [ ]:
# MODEL_NAME = 'PSMNet'
# MODEL_PATH = 'PSMNet\\pretrained\\pretrained_sceneflow_new.tar'
MODEL_NAME = 'RAFT-Stereo'
MODEL_PATH = "RAFT-Stereo/models/raftstereo-realtime.pth"
TEST_LEFT_IMG = "G:\\My Drive\\Dataset\\20250619-xi-testing\\stereo\\L\\pose\\L00000016.tif"
TEST_RIGHT_IMG = "G:\\My Drive\\Dataset\\20250619-xi-testing\\stereo\\R\\pose\\R00000016.tif"

# stereo_params is from SystemCalibration (npz file with K/D/R/T/R_L/R_R/P_L/P_R/Q)
stereo_params = np.load('stereo_params.npz')
IMG_SIZE = (894, 714)

In [ ]:
# Load model
model = utils.load_model(MODEL_NAME, MODEL_PATH)

In [ ]:
# Calculate Undistortion and Rectification Remapping
K_L = stereo_params['K_L']
D_L = stereo_params['D_L']
K_R = stereo_params['K_R']
D_R = stereo_params['D_R']
R_L = stereo_params['R_L']
R_R = stereo_params['R_R']
P_L = stereo_params['P_L']
P_R = stereo_params['P_R']
Q = stereo_params['Q']

mapL1, mapL2 = cv2.initUndistortRectifyMap(K_L, D_L, R_L, P_L, IMG_SIZE, cv2.CV_16SC2)
mapR1, mapR2 = cv2.initUndistortRectifyMap(K_R, D_R, R_R, P_R, IMG_SIZE, cv2.CV_16SC2)

In [ ]:
# ------------------------------- Main runner ------------------------------------
def run_visualizer(
    mode: str,                                    # "images" | "video"
    stereo_params: dict,                          # dict with K/D/R/T/R_L/R_R/P_L/P_R/Q
    left_map1, left_map2, right_map1, right_map2, # rectification remap
    model_name: str = "",                         # "RAFT-Stereo" | "PSMNet"
    display_mode: str = "overlay",                # "overlay" | "points_only" | "both"
    radius: int = 2,
    alpha: float = 0.6,
    tint_strength: float = 0.7,
    blur: int = 9,
    # images mode
    left_img_path: str = "",
    right_img_path: str = "",
    # video mode
    left_src: str = "",
    right_src: str = "",
    target_size: tuple = None,            # (w, h) or None
):
    """
    Visualize uL / uR either from a pair of images or from two live video streams.

    Dependencies in utils.py:
      - load_model(model_name, model_weights)             (already called outside)
      - compute_disparity(model_name, model, left, right, left_map1,left_map2,right_map1,right_map2)
      - generate_pointcloud(model_name, disp, left, left_map1,left_map2, Q)
      - transform_pointcloud(pcd_or_points, stereo_params)
      - overlay_pointcloud_image(img_bgr, u, radius, alpha, tint_strength, blur, mode)

    Notes:
      - For "video", left_src and right_src should be two independent inputs (camera indices or stream URIs).
      - Rectification maps are assumed to match IMG_SIZE. If your frames differ, we resize to IMG_SIZE first.
    """
    # -------------------------- images mode --------------------------------------
    if mode == "images":
        imgL = cv2.imread(left_img_path,  cv2.IMREAD_COLOR)
        imgR = cv2.imread(right_img_path, cv2.IMREAD_COLOR)
        assert imgL is not None and imgR is not None, "Failed to read input images."

        # Disparity (use utils to keep the pipeline consistent)
        disp = utils.compute_disparity(
            model_name, model,
            left_img_path, right_img_path,
            left_map1, left_map2, right_map1, right_map2
        )

        # Point cloud (rectified-left frame)
        pcd = utils.generate_pointcloud(
            model_name, disp, left_img_path, left_map1, left_map2, stereo_params['Q']
        )

        # Transform to original coordinate system
        pcd_rect_L, pcd_rect_R = utils.pcd_to_rectified_images(pcd, stereo_params, (894, 714))
        pcd_rect_L = cv2.flip(pcd_rect_L, 1)
        pcd_rect_R = cv2.flip(pcd_rect_R, 1)
        uL = utils.inverse_remap_rectified_to_original(pcd_rect_L, stereo_params, camera="left")
        uR = utils.inverse_remap_rectified_to_original(pcd_rect_R, stereo_params, camera="right")
        
        # Visualize using utils.overlay_pointcloud_image
        def _viz_one(win, img, u):
            vis = utils.overlay_pointcloud_image(
                img, u,
                radius=radius, alpha=alpha,
                tint_strength=tint_strength, blur=blur,
                mode=display_mode
            )
            if isinstance(vis, tuple):  # "both"
                cv2.imshow(f"{win} overlay", vis[0])
                cv2.imshow(f"{win} points",  vis[1])
            else:
                cv2.imshow(win, vis)

        _viz_one("uL", imgL, uL)
        _viz_one("uR", imgR, uR)

        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return
    
    # -------------------------- video mode ---------------------------------------
    elif mode == "video":
        # Open left/right video sources
        capL = cv2.VideoCapture(left_src)
        capR = cv2.VideoCapture(right_src)
        assert capL.isOpened() and capR.isOpened(), "Failed to open left/right video sources."

        # Read first frames
        okL, frameL0 = capL.read()
        okR, frameR0 = capR.read()
        assert okL and okR,  "Failed to read the first frame from left/right sources."

        # Decide a working size; maps were built for IMG_SIZE, so we’ll default to that
        if target_size is None:
            H0, W0 = frameL0.shape[:2]
            size_wh = (W0, H0)
        else:
            size_wh = target_size

        while True:
            t0 = time.time()

            # Better sync: grab then retrieve
            capL.grab(); capR.grab()
            okL, frameL = capL.retrieve()
            okR, frameR = capR.retrieve()
            if not (okL and okR):
                break

            # Disparity on frames (compute_disparity accepts ndarray frames)
            disp = utils.compute_disparity_frames(
                model_name, 
                model,
                left = frameL, right = frameR,              # pass ndarray frames
                left_map1 = left_map1, left_map2 = left_map2,
                right_map1 = right_map1, right_map2 = right_map2)
            
            # Point cloud using the left frame (function will remap internally)            
            pcd = utils.generate_pointcloud_frames(
                model_name, 
                disp, 
                left = frameL,                             # pass ndarray left frame
                left_map1 = left_map1, left_map2 = left_map2, 
                Q = Q)
            
            # Transform to original coordinate system
            pcd_rect_L, pcd_rect_R = utils.pcd_to_rectified_images(pcd, stereo_params, (894, 714))
            pcd_rect_L = cv2.flip(pcd_rect_L, 1)
            pcd_rect_R = cv2.flip(pcd_rect_R, 1)
            uL = utils.inverse_remap_rectified_to_original(pcd_rect_L, stereo_params, camera="left")
            uR = utils.inverse_remap_rectified_to_original(pcd_rect_R, stereo_params, camera="right")

            # Visualize on the raw frames
            def _viz(win, img, u):
                vis = utils.overlay_pointcloud_image(
                    img, u,
                    radius=radius, alpha=alpha,
                    tint_strength=tint_strength, blur=blur,
                    mode=display_mode
                )
                if isinstance(vis, tuple):  # "both"
                    cv2.imshow(f"{win} overlay", vis[0])
                    cv2.imshow(f"{win} points",  vis[1])
                else:
                    cv2.imshow(win, vis)
            
            _viz("uL", frameL, uL)
            _viz("uR", frameR, uR)

            # FPS display
            fps = 1.0 / max(time.time() - t0, 1e-6)
            try:
                cv2.setWindowTitle("uL", f"uL  ({fps:.1f} FPS)")
                cv2.setWindowTitle("uR", f"uR  ({fps:.1f} FPS)")
            except:
                pass

            k = cv2.waitKey(1) & 0xFF
            if k in (27, ord('q')):  # ESC / q
                break

        capL.release(); capR.release()
        cv2.destroyAllWindows()

In [ ]:
# ------------------------------- Example usage ------------------------------------
run_visualizer(
    mode="images",
    stereo_params=stereo_params,
    left_map1=mapL1, left_map2=mapL2, right_map1=mapR1, right_map2=mapR2,
    model_name=MODEL_NAME,
    left_img_path=TEST_LEFT_IMG,
    right_img_path=TEST_RIGHT_IMG,
    display_mode="both",    # "overlay" | "points_only" | "both"
    radius=2, alpha=0.6, tint_strength=0.7, blur=11
)

In [ ]:
# # ------------------------------- Example usage ------------------------------------
# run_visualizer(
#     mode="video",
#     stereo_params=stereo_params,
#     left_map1=mapL1, left_map2=mapL2, right_map1=mapR1, right_map2=mapR2,
#     model_name=MODEL_NAME,
#     left_src=0, right_src=1,                 # two independent live streams
#     display_mode="overlay",                  # "overlay" | "points_only" | "both"
#     radius=2, alpha=0.6, tint_strength=0.7, blur=11,
#     target_size=IMG_SIZE                     # or (W,H) to enforce a working size
# )